Mateo Tavares Trueba -- Actividad de Práctica

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

In [7]:
df=pd.read_csv('brain_tumor_dataset.csv')

In [8]:
df.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [9]:
num_features = [
    'Age', 
    'Tumor_Size', 
    'Survival_Rate', 
    'Tumor_Growth_Rate'
]
cat_features = [
    'Gender', 
    'Location', 
    'Histology', 
    'Stage', 
    'Symptom_1', 
    'Symptom_2', 
    'Symptom_3', 
    'Radiation_Treatment', 
    'Surgery_Performed', 
    'Chemotherapy', 
    'Family_History', 
    'MRI_Result', 
    'Follow_Up_Required'
]

target = df.columns[df.columns == 'Tumor_Type'].tolist()
x=df[num_features+cat_features]
y=df[target]



In [10]:
numerical_transformer=StandardScaler()
preprocesser=ColumnTransformer(transformers=[('num',numerical_transformer,num_features)])

In [11]:
model= SVC(kernel='linear', class_weight='balanced')

In [12]:
pipeline=Pipeline(steps=[
    ('preprocesser',preprocesser),
    ('model',model)

])
pipeline.fit(x,y)

c:\Users\mateo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Tumor_Size',
                                                   'Survival_Rate',
                                                   'Tumor_Growth_Rate'])])),
                ('model', SVC(class_weight='balanced', kernel='linear'))])

In [ ]:
pipeline.score(x,y)

In [22]:
def funcion_objetivo(X_hl) :
    model= SVC(kernel='linear', class_weight='balanced',C=X_hl,probability=True)
    pipeline=Pipeline(steps=[
    ('preprocesser',preprocesser),
    ('model',model)

    ])
    scores = cross_val_score(pipeline, x, y.values.ravel(), 
                               scoring='roc_auc_ovr', cv=10)
    meanscore = np.mean(scores)
    return meanscore

In [23]:
y1 = funcion_objetivo(0.001)
y2 = funcion_objetivo(1)
y3 = funcion_objetivo(10)
X_gauss= np.array([[.001],[1],[10]])
Y_scores = [y1, y2, y3]

kfolds = KFold(n_splits=10, shuffle=True, random_state=10)
Y_scores = np.array(Y_scores)
print("X_gauss =", X_gauss)
print("Y_scores =", Y_scores)
regresion = GaussianProcessRegressor(kernel=RBF(length_scale=1.0), random_state=10)


X_gauss = [[1.e-03]
 [1.e+00]
 [1.e+01]]
Y_scores = [0.50112361 0.49803993 0.49605916]


In [ ]:
# ====================================================================
# INICIO DEL CICLO DE OPTIMIZACIÓN BAYESIANA
# ====================================================================


max_ucb_value = 1.0 # Inicializamos el valor máximo en Y de los intervalos de confianza.
Y_max_real = Y_scores.max() # El mejor score real encontrado hasta ahora.
n_iteraciones = 0
max_iteraciones = 300 # Iteraciones del ciclo.

# 2. Preparar los arreglos para las nuevas observaciones
# Convertimos las listas iniciales a arreglos de numpy para facilitar la adición
X_data = np.array(X_gauss)
Y_data = Y_scores.copy() 

# --------------------------------------------------------------------
# CICLO PRINCIPAL
# --------------------------------------------------------------------

while (Y_max_real <= max_ucb_value) and (n_iteraciones < max_iteraciones):
    
    n_iteraciones += 1
    print(f"--- Iteración {n_iteraciones} ---")
    
    # A. AJUSTAR REGRESIÓN GAUSSIANA (Actualización del Modelo)
    # Se ajusta con TODOS los puntos (iniciales + nuevos)
    regresion.fit(X_data, Y_data)
    
    # Obtenemos la media y la varianza para los intervalos de confianza.
    n1_values = np.arange(0.001, 10, 0.01) 
    #n2_values = np.arange(5, 15, 1)
    #n1_grid, n2_grid = np.meshgrid(n1_values, n2_values)
    X_test_completo = n1_values.reshape([-1, 1])#np.c_[n1_grid.ravel(), n2_grid.ravel()]

    # FILTRADO CRÍTICO: Creamos una máscara para eliminar los puntos ya evaluados.
    # np.isin comprueba qué filas de X_test_completo YA están en X_data
    # ~ niega la máscara, seleccionando solo los que NO están en X_data
    is_not_evaluated = ~np.isin(X_test_completo, X_data, assume_unique=False).all(axis=1)
    
    # Los puntos candidatos reales a evaluar son solo los que no están en X_data
    X_test = X_test_completo[is_not_evaluated]

    # Si no quedan puntos para evaluar, terminamos el ciclo
    if len(X_test) == 0:
        print("No quedan puntos nuevos para explorar en X_test. Terminando.")
        break
    mu, sigma = regresion.predict(X_test, return_std=True)

    # C. CALCULAR UCB (intervalos de confianza)
    kappa = 1.96 
    ucb = mu + kappa * sigma
    
    # Obtener el valor UCB máximo
    max_ucb_value = np.max(ucb)
    print(f"Máximo UCB predicho en X_test: {max_ucb_value:.4f}")
    
    #Valor máximo de ucb (intervalos de confianza)
    idx_max_ucb = np.argmax(ucb)
    X_next = X_test[idx_max_ucb].reshape(1, -1)
    
    # E. EVALUAR EL PUNTO X_NEXT EN EL MUNDO REAL (Función Objetivo)
    Y_next_real = funcion_objetivo(X_next[0][0])
    
    print(f"  > Punto evaluado: {X_next[0]}, ROC AUC real: {Y_next_real:.4f}")
    
    # F. ACTUALIZAR LOS ARRAYS DE DATOS Y EL MÁXIMO REAL
    X_data = np.vstack([X_data, X_next]) # Añadir la X al set de entrenamiento
    Y_data = np.hstack([Y_data, Y_next_real]) # Añadir la Y al set de entrenamiento
    
    # Actualizar el máximo ROC AUC real encontrado hasta ahora
    Y_max_real = np.max(Y_data) 
    print(f"  > Máximo ROC AUC real hasta ahora: {Y_max_real:.4f}")
    
    print("-" * 50)
    
# ====================================================================
# FINAL DEL CICLO
# ====================================================================

print("\n!!! ÓPTIMIZACIÓN BAYESIANA COMPLETADA !!!")
print(f"La condición de parada se cumplió en la iteración {n_iteraciones}.")
print(f"El ROC AUC OVR máximo encontrado es: {Y_max_real:.4f}")


--- Iteración 1 ---
Máximo UCB predicho en X_test: 0.5011
